<a href="https://colab.research.google.com/github/atimad/Notebooks/blob/master/COVID_19_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using the CSSEGISandData/COVID-19 time series dataset, let's transform, visualize the data and do some analysis.

# COVID-19 Global Cases
## Load libraries

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#!pip install us
!pip install plotly --upgrade
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pylab import rcParams
from pandas.api.types import CategoricalDtype
from datetime import date, timedelta
import warnings
import folium
from folium.plugins import HeatMap
import pandas_profiling
import math
#import us

rcParams["figure.figsize"] = 20,9
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

     |████████████████████████████████| 7.1MB 2.6MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


# Data Import

In [0]:
# import data from github url, url1 -Confirmed, url2 -Deaths. url3 -Recovered
url1='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
url2='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
url3='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df1 = pd.read_csv(url1, error_bad_lines=False)
df2 = pd.read_csv(url2, error_bad_lines=False)
df3 = pd.read_csv(url3, error_bad_lines=False)
# display df1 -Confirmed df2 -Deaths df3 -Recovered
display(df1.head(), df2.head(), df3.head())

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,7,7,11,15,20,20,20,22,22,45,25,25,26,26,26,28,28,29,43,59,66,74,84,94,105,122,147,159,170,189,214,228,241,256,274,293,331,360,420,461,502,511,581,639,639
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,7,7,10,13,16,18,18,24,28,28,30,33,40,45,47,50,58,67,72,75,77,81,84,84,85,85,89,89,91,93,93,93,102,106,108,110,110,117,130,138,150,150,160,178,178
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,29,36,50,50,83,93,99,117,129,149,149


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,2,4,4,5,6,6,6,6,6,6,6,6,10,10,15,16
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,2,5,5,5,5,5,5,5,5,5,5,5,10,10,10,10,10,12,12,12,14,15,15,15,15,17,17,21,21,22,22,22,28,28,28,31,31,31,31,31,31,31,31,33,34,34
1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,9,9,9,9,12,12,12,13,18,18,22,22,22,22,22,22,22,22,32,32,32,43,43,43,46,76,76,76,101,118,118
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,9,15,15,17,18,18,24,29,34,34,37,37,51,51,53,62,62,62,72,72,78,78,78,78,78,78,78,78,78,96,96
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3,3,3,3,7,7,7,13,15,15,15,15,15,18,18,18,18,18,18,18,18,22,22,22,22,23,24,24,24,26,26


# **Data Manipulation**

In [0]:
# unpivot data frames: df1 (Confirmed) and name it df1u, df2 (Deaths) and name it df2u, df3 (Recovered) and name it df3u
df1u=pd.melt(df1, id_vars=['Province/State', 'Country/Region', 'Lat','Long'], var_name='DateTime', value_name='Confirmed')
df2u=pd.melt(df2, id_vars=['Province/State', 'Country/Region', 'Lat','Long'], var_name='DateTime', value_name='Deaths')
df3u=pd.melt(df3, id_vars=['Province/State', 'Country/Region', 'Lat','Long'], var_name='DateTime', value_name='Recovered')
# show unpivoted data frame for Confirmed, Deaths, Recovered
display(df1u.head(), df2u.head(), df3u.head())

,Province/State,Country/Region,Lat,Long,DateTime,Confirmed
0,NaN,Thailand,15.0000,101.0000,1/22/20,2
1,NaN,Japan,36.0000,138.0000,1/22/20,2
2,NaN,Singapore,1.2833,103.8333,1/22/20,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0


,Province/State,Country/Region,Lat,Long,DateTime,Deaths
0,NaN,Thailand,15.0000,101.0000,1/22/20,0
1,NaN,Japan,36.0000,138.0000,1/22/20,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0


,Province/State,Country/Region,Lat,Long,DateTime,Recovered
0,NaN,Thailand,15.0000,101.0000,1/22/20,0
1,NaN,Japan,36.0000,138.0000,1/22/20,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0


In [0]:
# Check number of NaN in data frame
display(df1u.info(), df2u.info(), df3u.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    15198 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null object
Confirmed         20706 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 970.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    15198 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null object
Deaths            20706 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 970.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    15198 non-null object
Country/Region    20706 no

None

None

None

In [0]:
# fill NaN with 0 for Confirmed, Deaths, Recovered
df1u['Confirmed'] = df1u['Confirmed'].fillna(0)
df2u['Deaths'] = df2u['Deaths'].fillna(0)
df3u['Recovered'] = df3u['Recovered'].fillna(0)
# fill NaN in Province/State column with data from Country/Region column
df3u['Province/State'] = df3u['Province/State'].fillna(df3u['Country/Region'])
df2u['Province/State'] = df2u['Province/State'].fillna(df2u['Country/Region'])
df1u['Province/State'] = df1u['Province/State'].fillna(df1u['Country/Region'])
# Display fill NaN for Confirmed, Deaths, Recovered
display(df1u.head(), df2u.head(), df3u.head(), df1u.info(), df2u.info(), df3u.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    20706 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null object
Confirmed         20706 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 970.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    20706 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null object
Deaths            20706 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 970.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    20706 non-null object
Country/Region    20706 no

,Province/State,Country/Region,Lat,Long,DateTime,Confirmed
0,Thailand,Thailand,15.0000,101.0000,1/22/20,2
1,Japan,Japan,36.0000,138.0000,1/22/20,2
2,Singapore,Singapore,1.2833,103.8333,1/22/20,0
3,Nepal,Nepal,28.1667,84.2500,1/22/20,0
4,Malaysia,Malaysia,2.5000,112.5000,1/22/20,0


,Province/State,Country/Region,Lat,Long,DateTime,Deaths
0,Thailand,Thailand,15.0000,101.0000,1/22/20,0
1,Japan,Japan,36.0000,138.0000,1/22/20,0
2,Singapore,Singapore,1.2833,103.8333,1/22/20,0
3,Nepal,Nepal,28.1667,84.2500,1/22/20,0
4,Malaysia,Malaysia,2.5000,112.5000,1/22/20,0


,Province/State,Country/Region,Lat,Long,DateTime,Recovered
0,Thailand,Thailand,15.0000,101.0000,1/22/20,0
1,Japan,Japan,36.0000,138.0000,1/22/20,0
2,Singapore,Singapore,1.2833,103.8333,1/22/20,0
3,Nepal,Nepal,28.1667,84.2500,1/22/20,0
4,Malaysia,Malaysia,2.5000,112.5000,1/22/20,0


None

None

None

In [0]:
# Change Data type for Columns DateTime, Confirmed, Deaths, and Recovered
df1u['DateTime'] = pd.to_datetime(df1u['DateTime'])
df2u['DateTime'] = pd.to_datetime(df2u['DateTime'])
df3u['DateTime'] = pd.to_datetime(df3u['DateTime'])
df1u['Confirmed']= pd.to_numeric(df1u['Confirmed'], downcast='integer')
df2u['Deaths']= pd.to_numeric(df2u['Deaths'], downcast='integer')
df3u['Recovered']= pd.to_numeric(df3u['Recovered'], downcast='integer')
display(df1u.info(), df2u.info(), df3u.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    20706 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null datetime64[ns]
Confirmed         20706 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(1), object(2)
memory usage: 889.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/State    20706 non-null object
Country/Region    20706 non-null object
Lat               20706 non-null float64
Long              20706 non-null float64
DateTime          20706 non-null datetime64[ns]
Deaths            20706 non-null int16
dtypes: datetime64[ns](1), float64(2), int16(1), object(2)
memory usage: 849.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 6 columns):
Province/Sta

None

None

None

In [0]:
# merge data frame df1u, df2u, df3u
merged_df1= pd.merge(df1u, df2u, how='left', left_on=['Province/State','DateTime'], right_on=['Province/State','DateTime'])
all_data= pd.merge(merged_df1, df3u,how='left', left_on=['Province/State','DateTime'], right_on=['Province/State','DateTime'])
all_data.head(), all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21318 entries, 0 to 21317
Data columns (total 14 columns):
Province/State      21318 non-null object
Country/Region_x    21318 non-null object
Lat_x               21318 non-null float64
Long_x              21318 non-null float64
DateTime            21318 non-null datetime64[ns]
Confirmed           21318 non-null int32
Country/Region_y    21318 non-null object
Lat_y               21318 non-null float64
Long_y              21318 non-null float64
Deaths              21318 non-null int16
Country/Region      21318 non-null object
Lat                 21318 non-null float64
Long                21318 non-null float64
Recovered           21318 non-null int32
dtypes: datetime64[ns](1), float64(6), int16(1), int32(2), object(4)
memory usage: 2.2+ MB


(  Province/State Country/Region_x    Lat_x  ...      Lat      Long  Recovered
 0       Thailand         Thailand  15.0000  ...  15.0000  101.0000          0
 1          Japan            Japan  36.0000  ...  36.0000  138.0000          0
 2      Singapore        Singapore   1.2833  ...   1.2833  103.8333          0
 3          Nepal            Nepal  28.1667  ...  28.1667   84.2500          0
 4       Malaysia         Malaysia   2.5000  ...   2.5000  112.5000          0
 
 [5 rows x 14 columns], None)

In [0]:
# Drop Colums 'Province/State_y', 'Country/Region_y', 'Province/State', 'Country/Region'

all_data.drop(columns=['Lat_y', 'Long_y', 'Lat', 'Long', 'Country/Region', 'Country/Region_y'], inplace=True)
all_data.head()

,Province/State,Country/Region_x,Lat_x,Long_x,DateTime,Confirmed,Deaths,Recovered
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


In [0]:
# Rename Columns Province/State_x to Province/State and Country/Region_x to Country/Region
all_data.rename(columns={"Lat_x": "Lat", "Long_x": "Long", "Country/Region_x": "Country/Region"}, inplace=True)
all_data['Confirmed'] = all_data['Confirmed'].fillna(0)
all_data['Deaths'] = all_data['Deaths'].fillna(0)
all_data['Recovered'] = all_data['Recovered'].fillna(0)
all_data['Recovered']= pd.to_numeric(all_data['Recovered'], downcast='integer')
all_data['Existing'] = all_data['Confirmed']-all_data['Deaths']-all_data['Recovered']
display(all_data.head(), all_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21318 entries, 0 to 21317
Data columns (total 9 columns):
Province/State    21318 non-null object
Country/Region    21318 non-null object
Lat               21318 non-null float64
Long              21318 non-null float64
DateTime          21318 non-null datetime64[ns]
Confirmed         21318 non-null int32
Deaths            21318 non-null int16
Recovered         21318 non-null int32
Existing          21318 non-null int32
dtypes: datetime64[ns](1), float64(2), int16(1), int32(3), object(2)
memory usage: 1.3+ MB


,Province/State,Country/Region,Lat,Long,DateTime,Confirmed,Deaths,Recovered,Existing
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2,0,0,2
1,Japan,Japan,36.0000,138.0000,2020-01-22,2,0,0,2
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0,0,0,0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0,0,0,0


None

In [0]:
# Show Last Update of COVID-19 Cases
# Create filter for DateTime to equal LastUpdate
LastUpdate=all_data['DateTime'].max()
LastUpdate_Filter=all_data['DateTime']==LastUpdate

In [0]:
# Create filtered data frame by LastUpdate_Filter
TotalCases_LastUpdate=all_data[LastUpdate_Filter]
TotalCases_LastUpdate.sort_values(by=['Confirmed'], inplace=True, ascending=False)
TotalCases_LastUpdate.reset_index(0, drop=True, inplace=True)
TotalCases_LastUpdate['Province/State'] = TotalCases_LastUpdate['Province/State'].fillna(TotalCases_LastUpdate['Country/Region'])
TotalCases_LastUpdate['Existing'] = TotalCases_LastUpdate['Confirmed']-TotalCases_LastUpdate['Deaths']-TotalCases_LastUpdate['Recovered']
TotalConf=TotalCases_LastUpdate['Confirmed'].sum()
TotalDeaths=TotalCases_LastUpdate['Deaths'].sum()
TotalRecovered=TotalCases_LastUpdate['Recovered'].sum()
TotalExisting=TotalCases_LastUpdate['Existing'].sum()
NCountries=TotalCases_LastUpdate['Country/Region'].nunique()
EstMortalityRate=TotalDeaths/TotalConf
#display(TotalCases_LastUpdate.head(), TotalCases_LastUpdate.info(), TotalCases_PreviousDay.head(),TotalCasesDelta.head())
print("Last Update", LastUpdate.strftime('%d %b %Y'), sep=": ")
print("Total Confirmed", '{:,}'.format(TotalConf), sep=": ")
print("Total Deaths", '{:,}'.format(TotalDeaths), sep=": ")
print("Total Recovered", '{:,}'.format(TotalRecovered), sep=": ")
print("Existing",'{:,}'.format(TotalExisting), sep=": ")
print("Estimated Mortality rate",'{:.2%}'.format(EstMortalityRate), sep=": " )

Last Update: 12 Mar 2020
Total Confirmed: 130,734
Total Deaths: 4,744
Total Recovered: 69,299
Existing: 56,691
Estimated Mortality rate: 3.63%


In [0]:
# Total Cases by Country/Region and Last Update
TotalCasesByLastUpdateCountry=TotalCases_LastUpdate.groupby(['Country/Region'], sort=False)['Confirmed', 'Deaths', 'Recovered', 'Existing'].sum()
TotalCasesByLastUpdateCountry=TotalCasesByLastUpdateCountry.reset_index()
print("Last Update", LastUpdate.strftime('%d %b %Y'), sep=": ")
display(TotalCasesByLastUpdateCountry.style.background_gradient(cmap='Set1_r'))

Last Update: 12 Mar 2020


,Country/Region,Confirmed,Deaths,Recovered,Existing
0,China,80932,3172,62901,14859
1,Italy,12462,827,1045,10590
2,Iran,10075,429,2959,6687
3,"Korea, South",7869,66,333,7470
4,France,2284,48,12,2224
5,Spain,2277,55,183,2039
6,Germany,2078,3,25,2050
7,Norway,702,0,1,701
8,Cruise Ship,2784,14,650,2120
9,Switzerland,652,4,4,644


In [0]:
# Total Cases by Country/Region, Province/State, and Last Update
TotalCasesByLastUpdateProvince=TotalCases_LastUpdate.groupby(['Country/Region', 'Province/State'],sort=False)['Confirmed', 'Deaths', 'Recovered','Existing'].sum()
TotalCasesByLastUpdateProvince=TotalCasesByLastUpdateProvince.reset_index()
print("Last Update", LastUpdate.strftime('%d %b %Y'), sep=": ")
display(TotalCasesByLastUpdateProvince.style.background_gradient(cmap='Set1_r'))

Last Update: 12 Mar 2020


In [0]:
# Total Cases by Date
TotalCases=all_data.groupby(['DateTime']).agg({"Confirmed": sum, 'Deaths': sum, 'Recovered': sum, 'Existing': sum})
TotalCases=TotalCases.reset_index()
TotalCases['deltaConfirmed']=TotalCases['Confirmed'].diff().fillna(0)
TotalCases['deltaDeaths']=TotalCases['Deaths'].diff().fillna(0)
TotalCases['deltaRecovered']=TotalCases['Recovered'].diff().fillna(0)
TotalCases['deltaExisting']=TotalCases['Existing'].diff().fillna(0)
# Total Cases Excluding China by Date
ExChina=all_data[all_data['Country/Region']!='China']
ExChina=ExChina.reset_index()
TotalCasesExChina=ExChina.groupby(['DateTime']).agg({"Confirmed": sum, 'Deaths': sum, 'Recovered': sum, 'Existing': sum})
TotalCasesExChina=TotalCasesExChina.reset_index()
TotalCasesExChina['deltaConfirmed']=TotalCasesExChina['Confirmed'].diff().fillna(0)
TotalCasesExChina['deltaDeaths']=TotalCasesExChina['Deaths'].diff().fillna(0)
TotalCasesExChina['deltaRecovered']=TotalCasesExChina['Recovered'].diff().fillna(0)
TotalCasesExChina['deltaExisting']=TotalCasesExChina['Existing'].diff().fillna(0)
# Total Cases by Province and Date
TotalCasesByProvince=all_data.groupby(['Province/State', 'DateTime']).agg({"Confirmed": sum, 'Deaths': sum, 'Recovered': sum, 'Existing': sum})
TotalCasesByProvince=TotalCasesByProvince.reset_index()
TotalCasesByProvince['Confirmed_Delta']=TotalCasesByProvince.sort_values(['Province/State', 'DateTime']).groupby('Province/State')['Confirmed'].diff().fillna(0)
TotalCasesByProvince['Deaths_Delta']=TotalCasesByProvince.sort_values(['Province/State', 'DateTime']).groupby('Province/State')['Deaths'].diff().fillna(0)
TotalCasesByProvince['Recovered_Delta']=TotalCasesByProvince.sort_values(['Province/State', 'DateTime']).groupby('Province/State')['Recovered'].diff().fillna(0)
TotalCasesByProvince['Existing_Delta']=TotalCasesByProvince.sort_values(['Province/State', 'DateTime']).groupby('Province/State')['Existing'].diff().fillna(0)
# Total Cases by Country and Date
TotalCasesByCountry=all_data.groupby(['Country/Region', 'DateTime']).agg({"Confirmed": sum, 'Deaths': sum, 'Recovered':sum, 'Existing': sum})
TotalCasesByCountry=TotalCasesByCountry.reset_index()
TotalCasesByCountry['Confirmed_Delta']=TotalCasesByCountry.sort_values(['Country/Region', 'DateTime']).groupby('Country/Region')['Confirmed'].diff().fillna(0)
TotalCasesByCountry['Deaths_Delta']=TotalCasesByCountry.sort_values(['Country/Region', 'DateTime']).groupby('Country/Region')['Deaths'].diff().fillna(0)
TotalCasesByCountry['Recovered_Delta']=TotalCasesByCountry.sort_values(['Country/Region', 'DateTime']).groupby('Country/Region')['Recovered'].diff().fillna(0)
TotalCasesByCountry['Existing_Delta']=TotalCasesByCountry.sort_values(['Country/Region', 'DateTime']).groupby('Country/Region')['Existing'].diff().fillna(0)

#display(TotalCases.head(), TotalCasesByProvince.head(), TotalCases.info(), TotalCasesByProvince.info())
cl=sns.light_palette("red", as_cmap=True)
display(TotalCases[['DateTime','Existing','Deaths','Recovered']].style.background_gradient(cmap=cl).set_caption('Heat Map of Total Existing, Deaths, and Recovered by Date'))
#display(TotalCasesByCountry[['Country/Region','DateTime','Existing','Deaths','Recovered']].style.background_gradient(cmap=cl).set_caption('Heat Map of Total Existing, Deaths, and Recovered by Country and Date'))
display(TotalCasesExChina.style.background_gradient(cmap=cl))

,DateTime,Existing,Deaths,Recovered
0,2020-01-22 00:00:00,510,17,28
1,2020-01-23 00:00:00,605,18,30
2,2020-01-24 00:00:00,879,26,36
3,2020-01-25 00:00:00,1353,42,39
4,2020-01-26 00:00:00,2010,56,52
5,2020-01-27 00:00:00,2784,82,61
6,2020-01-28 00:00:00,5340,131,107
7,2020-01-29 00:00:00,5907,133,126
8,2020-01-30 00:00:00,7920,171,143
9,2020-01-31 00:00:00,9492,213,222


,DateTime,Confirmed,Deaths,Recovered,Existing,deltaConfirmed,deltaDeaths,deltaRecovered,deltaExisting
0,2020-01-22 00:00:00,7,0,0,7,0,0,0,0
1,2020-01-23 00:00:00,10,0,0,10,3,0,0,3
2,2020-01-24 00:00:00,21,0,0,21,11,0,0,11
3,2020-01-25 00:00:00,28,0,0,28,7,0,0,7
4,2020-01-26 00:00:00,43,0,3,40,15,0,3,12
5,2020-01-27 00:00:00,50,0,3,47,7,0,0,7
6,2020-01-28 00:00:00,69,0,6,63,19,0,3,16
7,2020-01-29 00:00:00,79,0,6,73,10,0,0,10
8,2020-01-30 00:00:00,93,0,8,85,14,0,2,12
9,2020-01-31 00:00:00,125,0,8,117,32,0,0,32


# **Data Visualization**

In [0]:
# Time Series Visual for Confirmed, Recovered, and Deaths 
fig = go.Figure()
fig.add_trace(go.Scatter(x = TotalCases.DateTime, y = TotalCases['Confirmed'], name = "Total Confirmed", line_color = 'orange', mode = 'lines+markers', marker = dict(size = 9, symbol = 'circle')))
fig.add_trace(go.Scatter(x = TotalCases.DateTime, y = TotalCases['Recovered'], name = "Total Recovered", line_color = 'green', mode = 'lines+markers', marker = dict(size = 9, symbol = 'circle')))
fig.add_trace(go.Scatter(x = TotalCases.DateTime, y = TotalCases['Deaths'], name = "Total Deaths", line_color = 'firebrick', mode = 'lines+markers', marker = dict(size = 9, symbol = 'circle')))
fig.update_layout(title_text = 'Time Series Total Cases',
  xaxis_rangeslider_visible = True,
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()
# Time Series Visual for Existing Cases
fig = go.Figure()
fig.add_trace(go.Scatter(x = TotalCases.DateTime, y = TotalCases['Existing'], name = "Total Existing", line_color = 'blue', mode = 'lines+markers', marker = dict(size = 9, symbol = 'circle')))
fig.update_layout(title_text = 'Time Series Total Existing', showlegend=True,
  xaxis_rangeslider_visible = True,
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()
# Time Series Visual for Existing Cases Except China
fig = go.Figure()
fig.add_trace(go.Scatter(x = TotalCasesExChina.DateTime, y = TotalCasesExChina['Existing'], name = "Total Existing Excluding China", line_color = 'violet', mode = 'lines+markers', marker = dict(size = 9, symbol = 'circle')))
fig.update_layout(title_text = 'Time Series Total Existing Excluding China', showlegend=True,
  xaxis_rangeslider_visible = True,
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()


In [0]:
# Visual Display of Day to Day increase in new cases
fig = go.Figure()
fig.add_trace(go.Bar(x = TotalCases.DateTime, y = TotalCases['deltaConfirmed'], name = "Total Confirmed Delta", marker_color='orange') )
fig.add_trace(go.Bar(x = TotalCases.DateTime, y = TotalCases['deltaDeaths'], name = "Total Deaths Delta", marker_color='firebrick') )
fig.add_trace(go.Bar(x = TotalCases.DateTime, y = TotalCases['deltaRecovered'], name = "Total Recovered Delta", marker_color='green') )
fig.update_layout(title_text = 'Total Cases Delta',
  xaxis_rangeslider_visible = True,
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [0]:
# Day to Day increase in Confirmed Cases
fig = px.bar(TotalCasesByCountry, x="DateTime", y="Confirmed_Delta", color='Country/Region', orientation='v', height=600,
             title='Day to Day Increase in Confirmed Cases', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.update_layout(
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()
# Day to Day increase in Recovered Cases
fig = px.bar(TotalCasesByCountry, x="DateTime", y="Recovered_Delta", color='Country/Region', orientation='v', height=600,
             title='Day to Day Increase in Total Recovered Cases', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.update_layout(
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()
# Day to Day increase in Existing Cases
fig = px.bar(TotalCasesByCountry, x="DateTime", y="Existing_Delta", color='Country/Region', orientation='v', height=600,
             title='Day to Day Increase in Existing Cases', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.update_layout(
  paper_bgcolor='rgba(233,233,233,233)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()
#Countries with confirmed cases
fig = px.choropleth(TotalCasesByLastUpdateCountry, locations="Country/Region", 
                    locationmode='country names', color="Confirmed", 
                    hover_name="Country/Region", range_color=[1,100], 
                    color_continuous_scale="orrd", 
                    title='Countries with Confirmed Cases')
fig.update_layout(
    paper_bgcolor='rgba(233,233,233,233)', 
    plot_bgcolor='rgba(0,0,0,0)', 
    coloraxis_showscale=False,
    title_x=0.5,
    margin={"r":0,"t":50,"l":0,"b":20})
fig.update_geos(resolution=110,
                projection_type="natural earth")
fig.show()
#US states by last update
USAstates=TotalCasesByLastUpdateProvince[TotalCasesByLastUpdateProvince['Country/Region']=='US']
USAstates['Province/State'] = USAstates['Province/State'].astype(str)
USAstates[['StateName2', 'StateCode2']]=USAstates['Province/State'].str.split(',', expand=True)
#Dictionary of US state codes thank to @rogerallen on github
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Diamond Princess': 'Ship1',
    'Grand Princess': 'Ship2',
}
#Creating data frame for US States Map
USAstates['StateCode']= np.where(USAstates['StateCode2'].isnull(), USAstates['StateName2'].map(us_state_abbrev), USAstates['StateCode2'])
inverted_dict = dict( map(reversed, us_state_abbrev.items() ) )
USAstates['StateCode']=USAstates['StateCode'].str.strip()
USAstates['StateName']= USAstates['StateCode'].map(inverted_dict)
USAstatesGroup=USAstates.groupby(['StateName', 'StateCode'],sort=False).agg({"Confirmed": sum, 'Deaths': sum, 'Recovered': sum, 'Existing': sum})
USAstatesGroup=USAstatesGroup.reset_index()
fig = px.choropleth(USAstatesGroup, locations="StateCode", 
                    locationmode='USA-states', color="Confirmed",scope='usa', 
                    hover_name="StateName", range_color=[1,100], 
                    color_continuous_scale="orrd", 
                    title='US States with Confirmed Cases')
fig.update_layout(
    paper_bgcolor='rgba(233,233,233,233)', 
    plot_bgcolor='rgba(0,0,0,0)', 
    coloraxis_showscale=False,
    title_x=0.5,
    margin={"r":0,"t":50,"l":0,"b":20})
fig.update_geos(resolution=110)
fig.show()


In [0]:
map1 = folium.Map(location=[30, 100], tiles = "CartoDB dark_matter", zoom_start=4.2)
TotalCases_LastUpdate['Confirmed']= TotalCases_LastUpdate['Confirmed'].astype(float)
TotalCases_LastUpdate['Deaths']= TotalCases_LastUpdate['Deaths'].astype(float)
TotalCases_LastUpdate['Recovered']= TotalCases_LastUpdate['Recovered'].astype(float)
for i in range(0,len(TotalCases_LastUpdate)):
   folium.Circle(
      location=[TotalCases_LastUpdate.iloc[i]['Lat'], TotalCases_LastUpdate.iloc[i]['Long']],
      popup=
              TotalCases_LastUpdate.iloc[i]['Province/State']+ 
       " Total Confirmed: "+str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Confirmed']))+
       " Total Deaths: "+ str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Deaths']))+
       " Total Recovered: "+ str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Recovered']))+
       " Total Existing: "+ str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Existing'])),
      radius=(math.sqrt(TotalCases_LastUpdate.iloc[i]['Confirmed'])*1200+2 ),
      tooltip = '<li><bold>Country : '+str(TotalCases_LastUpdate.iloc[i]['Country/Region'])+
                '<li><bold>Province : '+str(TotalCases_LastUpdate.iloc[i]['Province/State'])+
                '<li><bold>Confirmed : '+str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Confirmed']))+
                '<li><bold>Deaths : '+str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Deaths']))+
                '<li><bold>Recovered : '+str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Recovered']))+
                '<li><bold>Existing : '+str("{:,.0f}".format(TotalCases_LastUpdate.iloc[i]['Existing'])),
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(map1)
folium.TileLayer('openstreetmap').add_to(map1)
folium.TileLayer('Stamen Terrain').add_to(map1)
folium.TileLayer('Stamen Toner').add_to(map1)
folium.TileLayer('stamenwatercolor').add_to(map1)
folium.TileLayer('cartodbpositron').add_to(map1)
folium.LayerControl().add_to(map1)

map1